## 공용 코드

In [1]:
# 파이썬
# ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os

# 깔끔한 그래프 출력을 위해 %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
# 이미지를 저장할 디렉토리 생성
os.makedirs(IMAGES_PATH, exist_ok=True)

# 이미지 저장
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

from matplotlib import font_manager, rc
import platform

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
    
    
mpl.rcParams['axes.unicode_minus'] = False
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

# 그래픽 출력을 좀 더 고급화하기 위한 라이브러리
import seaborn as sns

# 과학 기술 통계 라이브러리
import scipy as sp
from scipy import stats

# 사이킷런 ≥0.20 필수
# 0.20 이상 버전에서 데이터 변환을 위한 Transformer 클래스가 추가됨
import sklearn
assert sklearn.__version__ >= "0.20"

# 노트북 실행 결과를 동일하게 유지하기 위해 시드 고정
# 데이터를 분할할 때 동일한 분할을 만들어 냄
np.random.seed(21)

## 텍스트 군집

### 디렉토리 내에서 .data 로 끝나는 파일 모두 읽기

In [2]:
# data/OpinosisDataset/topics 디렉토리에서 데이터 파일 전부 읽기

import glob, os
import platform

# 디렉토리의 이름(경로)을 생성
path_name = ""
if platform.system() == 'Darwin':
    path_name = './data/OpinosisDataset/topics'
elif platform.system() == 'Windows':
    path_name = '.\\data\\OpinosisDataset\\topics'

# 디렉토리 안의 모든 파일 이름을 list 로 생성
all_file_name = glob.glob(os.path.join(path_name, "*.data"))
print(all_file_name[:5])

['.\\data\\OpinosisDataset\\topics\\accuracy_garmin_nuvi_255W_gps.txt.data', '.\\data\\OpinosisDataset\\topics\\bathroom_bestwestern_hotel_sfo.txt.data', '.\\data\\OpinosisDataset\\topics\\battery-life_amazon_kindle.txt.data', '.\\data\\OpinosisDataset\\topics\\battery-life_ipod_nano_8gb.txt.data', '.\\data\\OpinosisDataset\\topics\\battery-life_netbook_1005ha.txt.data']


In [3]:
# 파일의 이름을 저장할 list 
file_name_list = []
# 파일의 내용을 저장할 list
opinion_text = []

# 파일의 경로를 순회하면서 파일의 내용을 읽어서 하나로 만들기

# 순회하면서 파일의 내용 읽기
for file in all_file_name:
    # 파일 이름에 해당하는 파일 읽기
    df = pd.read_table(file, index_col = None, header = 0, encoding = 'latin1')
    
    # 파일의 이름만 추출 - 파일 주소에서 가장 뒤가 파일 이름
    filename_ = file.split('\\')[-1]
    # 확장자에 사용하는 . 을 기준으로 가장 앞이 파일의 이름
    filename = filename_.split('.')[0]
    
    # 파일 이름과 내용을 리스트에 저장
    file_name_list.append(filename)
    opinion_text.append(df.to_string())
    
#print(file_name_list[:5])
#print(opinion_text[1])

# 파일 이름과 내용으로 DataFrame 을 생성
document_df = pd.DataFrame({'filename' : file_name_list,'opinion_text' : opinion_text})
document_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   filename      51 non-null     object
 1   opinion_text  51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


### 피처 벡터화

In [4]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 텍스트에서 구두점 제거

# 마침표, 느낌표 등에 해당하는 구두점의 ord
remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)
#print(remove_punc_dict)
# 인스턴스 생성
lemmar = WordNetLemmatizer()

# 토큰화하는 함수
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]
# 소문자로 바꿔서 토큰화하고 어근을 찾아오는 함수
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

# 피처 벡터화 적용
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english',
                            ngram_range = (1, 2), min_df = 0.05, max_df = 0.9)
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

# 피처 벡터화의 결과 확인
print(feature_vect)
# 문자에 어떤 단어가 어느 정도의 가중치를 가지고 있는지를
# 희소 행렬로 생성
# TF-IDF 방식이기 때문에 갯수가 아닌 가중치로 표현

C:\Users\USER\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


  (0, 1467)	0.02328103871534355
  (0, 385)	0.02328103871534355
  (0, 1472)	0.05108217042656528
  (0, 2656)	0.02328103871534355
  (0, 1359)	0.021823791182120654
  (0, 725)	0.02328103871534355
  (0, 4371)	0.021823791182120654
  (0, 4597)	0.017985227788980915
  (0, 4277)	0.02063313325216223
  (0, 1470)	0.019626446316331432
  (0, 1806)	0.019626446316331432
  (0, 4044)	0.02328103871534355
  (0, 1469)	0.015099822028591765
  (0, 2471)	0.02328103871534355
  (0, 2175)	0.021823791182120654
  (0, 4195)	0.021823791182120654
  (0, 4200)	0.019626446316331432
  (0, 234)	0.02328103871534355
  (0, 4045)	0.02328103871534355
  (0, 4096)	0.04126626650432446
  (0, 1535)	0.04364758236424131
  (0, 1998)	0.02328103871534355
  (0, 158)	0.021823791182120654
  (0, 1166)	0.02328103871534355
  (0, 3134)	0.021823791182120654
  :	:
  (50, 2030)	0.016585088556344184
  (50, 2039)	0.00910671917345855
  (50, 1657)	0.009756197200387806
  (50, 1323)	0.009446650467709946
  (50, 3298)	0.02211345140845891
  (50, 3434)	0.0173

### 군집 알고리즘 수행

In [5]:
# 군집 알고리즘들 중 K-Means 적용
from sklearn.cluster import KMeans

# 지정한 갯수로 클러스터링을 수행 - 8개의 군집 생성
km_cluster = KMeans(n_clusters = 8, max_iter = 1000, random_state = 21)
km_cluster.fit(feature_vect)

# 군집의 결과인 라벨(숫자) 가져오기
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 원본 데이터에 분류된 라벨을 더하기
document_df['cluster_labels'] = cluster_label
print(document_df.head())

C:\Users\USER\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                         filename  \
0   accuracy_garmin_nuvi_255W_gps   
1  bathroom_bestwestern_hotel_sfo   
2      battery-life_amazon_kindle   
3      battery-life_ipod_nano_8gb   
4     battery-life_netbook_1005ha   

                                        opinion_text  cluster_labels  
0                                                ...               7  
1                                                ...               0  
2                                                ...               6  
3                                                ...               6  
4                                                ...               6  


In [6]:
# 같은 군집에 속한 파일 이름 확인

# 같은 군집의 파일 이름들을 가져온 다음 filename 순으로 정렬
document_df[document_df['cluster_labels'] == 1].sort_values(by = 'filename')

,filename,opinion_text,cluster_labels
11,features_windows7,...,1
19,keyboard_netbook_1005ha,...,1
34,screen_garmin_nuvi_255W_gps,...,1
35,screen_ipod_nano_8gb,...,1
36,screen_netbook_1005ha,...,1
41,size_asus_netbook_1005ha,...,1
44,speed_windows7,...,1
49,video_ipod_nano_8gb,...,1


In [7]:
# 군집의 갯수 조절 - 8개가 아니라 3개

km_cluster = KMeans(n_clusters = 3, max_iter = 1000, random_state = 21)
km_cluster.fit(feature_vect)

cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

document_df['cluster_labels_for_3'] = cluster_label

# 클러스터별로 정렬
document_df.sort_values(by = 'cluster_labels_for_3')

print(document_df.head())

C:\Users\USER\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                         filename  \
0   accuracy_garmin_nuvi_255W_gps   
1  bathroom_bestwestern_hotel_sfo   
2      battery-life_amazon_kindle   
3      battery-life_ipod_nano_8gb   
4     battery-life_netbook_1005ha   

                                        opinion_text  cluster_labels  \
0                                                ...               7   
1                                                ...               0   
2                                                ...               6   
3                                                ...               6   
4                                                ...               6   

   cluster_labels_for_3  
0                     1  
1                     2  
2                     1  
3                     1  
4                     1  


### 군집을 생성한(분류한) 핵심 단어 확인

In [8]:
# 클러스터의 핵심 단어를 찾는 함수
# 모델 종류, 클러스터의 수, 몇 개의 단어를 찾을지 등을 입력
def get_cluster_center(cluster_model, cluster_data, 
                       feature_names, cluster_num, top_n_features = 10):
    cluster_details = {}
    
    # 군집 중심과의 거리가 먼 단어 순서대로 정렬해서 저장
    # cluster_model.cluster_centers_ 가 중심점의 좌료
    centroid_feature_order_far = cluster_model.cluster_centers_.argsort()[:, ::-1]
    
    # 
    for clust_num in range(cluster_num):
        cluster_details[clust_num] = {}
        cluster_details[clust_num]['cluster'] = clust_num
        
        # 지정한 갯수만큼 중요 피처 추출
        top_feature_indexs = centroid_feature_order_far[clust_num, :top_n_features]
        # 저장되어 있던 단어를 반영
        top_features = [feature_names[idx] for idx in top_feature_indexs]
        
        # 중심점과의 거리 저장
        top_feature_values = cluster_model.cluster_centers_[clust_num,
                                            top_feature_indexs].tolist()
        cluster_details[clust_num]['top_features'] = top_features
        cluster_details[clust_num]['top_feature_values'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_labels'] == clust_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[clust_num]['filenames'] = filenames
        
    return cluster_details
        
        

In [9]:
# 클러스터 별 핵심 단어를 출력하는 함수
def print_cluster_center(cluster_details):
    for clust_num, cluster_detail in cluster_details.items():
        print('### ', clust_num, ' ###')
        print('핵심 단어 :', cluster_detail['top_features'])
        print('파일명 :', cluster_detail['filenames'])
    

In [10]:
# 피처 이름을 전부 가져오기

feature_names = tfidf_vect.get_feature_names_out()
#print(feature_names)

# 클러스터에 대한 자세한 정보 가져오기
cluster_details = get_cluster_center(cluster_model = km_cluster,
                                     cluster_data = document_df,
                                     feature_names = feature_names,
                                     cluster_num = 3,
                                     top_n_features = 10)

# 클러스터에 대한 정보 출력
print_cluster_center(cluster_details)
# 결과 - 해당 파일들에는 포함되는 단어이지만 다른 파일에는 없는 단어들

###  0  ###
핵심 단어 : ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
파일명 : ['bathroom_bestwestern_hotel_sfo', 'rooms_bestwestern_hotel_sfo', 'rooms_swissotel_chicago', 'room_holiday_inn_london']
###  1  ###
핵심 단어 : ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
파일명 : ['features_windows7', 'keyboard_netbook_1005ha', 'screen_garmin_nuvi_255W_gps', 'screen_ipod_nano_8gb', 'screen_netbook_1005ha', 'size_asus_netbook_1005ha', 'speed_windows7', 'video_ipod_nano_8gb']
###  2  ###
핵심 단어 : ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
파일명 : ['gas_mileage_toyota_camry_2007', 'mileage_honda_accord_2008', 'transmission_toyota_camry_2007']


## 문장의 유사도 측정

### 코사인 유사도를 구하는 알고리즘

In [11]:
# 코사인 유사도를 구하는 함수
def cos_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    l2_norm = (np.sqrt(sum(np.square(vector1))) * np.sqrt(sum(np.square(vector2))))
    similarity = dot_product / l2_norm
    
    return similarity

In [12]:
# 샘플 데이터 생성
document_list = ['i need to go home', 'i need to have some',
                'i need to get some things']

# 전처리

tfidf_vect = TfidfVectorizer()
feature_vect = tfidf_vect.fit_transform(document_list)
print(feature_vect[0])
print('####')
print(feature_vect[2])
# 결과가 희소 행렬 형태이므로 아직은 거리를 계산할 수 없음
# 거리를 계산하려면 서로 구조가 같아야 하지만 문장 별로 피처의 수가 다름

# 희소 행렬을 밀집 행렬로 변환 - 서로 구조가 같아짐
feature_vect_dense = feature_vect.todense()
print(feature_vect_dense[0])
print(feature_vect_dense[2])

  (0, 3)	0.6088450986844796
  (0, 1)	0.6088450986844796
  (0, 7)	0.35959372325985667
  (0, 4)	0.35959372325985667
####
  (0, 6)	0.55249004708441
  (0, 0)	0.55249004708441
  (0, 5)	0.42018292148905534
  (0, 7)	0.3263095219528963
  (0, 4)	0.3263095219528963
[[0.    0.609 0.    0.609 0.36  0.    0.    0.36 ]]
[[0.552 0.    0.    0.    0.326 0.42  0.552 0.326]]


In [13]:
# 문장 사이의 거리 계산

# 거리 계산을 위해 1차원으로 변환
vect1 = np.array(feature_vect_dense[0]).reshape(-1, )
vect2 = np.array(feature_vect_dense[1]).reshape(-1, )
vect3 = np.array(feature_vect_dense[2]).reshape(-1, )

print('문장 1과 2의 유사도 :', cos_similarity(vect1, vect2))
print('문장 1과 3의 유사도 :', cos_similarity(vect1, vect3))
print('문장 2과 3의 유사도 :', cos_similarity(vect2, vect3))

문장 1과 2의 유사도 : 0.28155035771770787
문장 1과 3의 유사도 : 0.23467771186837183
문장 2과 3의 유사도 : 0.4673070015402796


### sklearn 을 활용해서 코사인 유사도 구하기

In [14]:
# sklearn 의 코사인 유사도 계산 API 를 사용해서 유사도 계산

from sklearn.metrics.pairwise import cosine_similarity

# 밀집 행렬로 변환하지 않고 희소 행렬 상태에서도 계산 가능
similarity_simple_pair = cosine_similarity(feature_vect[0],
                                          feature_vect)
print(similarity_simple_pair)
# 결과 - [[1.    0.282 0.235]]
# 자기 자신과는 결과가 1

[[1.    0.282 0.235]]


### 문서 군집의 코사인 유사도 확인

In [18]:
tfidf_vect = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english',
                            ngram_range = (1, 2), min_df = 0.05, max_df = 0.9)
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

km_cluster = KMeans(n_clusters = 8, max_iter = 1000, random_state = 21)
km_cluster.fit(feature_vect)

cluster_label = km_cluster.labels_
# 각 군집의 centroid
cluster_centers = km_cluster.cluster_centers_

# 이전의 문서 데이터 다시 사용
document_df['cluster_labels'] = cluster_label
print(cluster_centers)

[[0.    0.    0.005 ... 0.    0.002 0.001]
 [0.009 0.    0.    ... 0.01  0.    0.   ]
 [0.    0.003 0.    ... 0.    0.    0.   ]
 ...
 [0.006 0.    0.    ... 0.    0.    0.   ]
 [0.015 0.    0.    ... 0.    0.    0.   ]
 [0.011 0.    0.    ... 0.014 0.    0.   ]]


In [19]:
# 1번 클러스터의 문서들 간 코사인 유사도 확인

# 1번 클러스터의 인덱스 가져오기
hotel_indexs = document_df[document_df['cluster_labels'] == 1].index
#print('1번 클러스터의 인덱스 :', hotel_indexs)

# 유사도 계산 - 같은 그룹 내에서 유사도 계산
similarity_pair = cosine_similarity(feature_vect[hotel_indexs[0]],
                                    feature_vect[hotel_indexs])
print(similarity_pair)
# 그룹과 관계 없이 유사도 계산
similarity_pair = cosine_similarity(feature_vect[hotel_indexs[0]],
                                    feature_vect[:10])
print(similarity_pair)

# 결과 - 동일 군집 내에서는 유사도가 높게 나타나지만 다른 군집에 대해서는
# 낮은 유사도를 보임

[[1.    0.037 0.071 0.121 0.073 0.071 0.304 0.064]]
[[0.037 0.01  0.04  0.036 0.035 0.028 0.048 0.038 0.062 0.068]]


### 한글 문장의 유사도 측정

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
# TfidfVectorizer 로 바꾸거나 Okt 대신 Twitter 를 사용해도 무방함

okt = Okt()
vectorizer = CountVectorizer(min_df = 0.05)

# 문장 생성
contents = ['오늘은 날씨가 매우 좋다', '오늘도 날씨가 덥다',
           '오늘은 목요일이다', '서울 날씨는 맑음',
           '오늘 점심 뭐 먹지']

# 한글 문장 토큰화
contents_tokens = [okt.morphs(row) for row in contents]
print(contents_tokens)



[['오늘', '은', '날씨', '가', '매우', '좋다'], ['오늘', '도', '날씨', '가', '덥다'], ['오늘', '은', '목요일', '이다'], ['서울', '날씨', '는', '맑음'], ['오늘', '점심', '뭐', '먹지']]


In [34]:
# 토큰화된 결과를 가지고 다시 문장을 생성
# 피처 벡터화를 적용하기 위해서

contents_for_vect = []

# 토큰 단위로 구분된 문장을 생성
for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence += ' ' + word
    contents_for_vect.append(sentence)
    
print(contents_for_vect)

[' 오늘 은 날씨 가 매우 좋다', ' 오늘 도 날씨 가 덥다', ' 오늘 은 목요일 이다', ' 서울 날씨 는 맑음', ' 오늘 점심 뭐 먹지']


In [40]:
# 피처 벡터화

X = vectorizer.fit_transform(contents_for_vect)
# 피처 확인
print(vectorizer.get_feature_names_out())

# 피처 벡터화가 된 후의 문장 결과 확인
print(X.toarray().transpose())
print(X.toarray())

['날씨' '덥다' '맑음' '매우' '먹지' '목요일' '서울' '오늘' '이다' '점심' '좋다']
[[1 1 0 1 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [1 1 1 0 1]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [1 0 0 0 0]]
[[1 0 0 1 0 0 0 1 0 0 1]
 [1 1 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 1 1 0 0]
 [1 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 1 0]]


In [41]:
# 유사도를 측정할 문장 생성
new_content = ['오늘 점심 이후 오후 서울의 날씨는 매우 덥다']
new_content_tokens = [okt.morphs(row) for row in new_content]

# 토큰화된 문장 조립
new_content_for_vect = []
for content in new_content_tokens:
    sentence = ''
    for word in content:
        sentence += ' ' + word
    new_content_for_vect.append(sentence)

print(new_content_for_vect)


[' 오늘 점심 이후 오후 서울 의 날씨 는 매우 덥다']


In [42]:
# 테스트 문장을 피처 벡터화

new_content_vect = vectorizer.transform(new_content_for_vect)
print(new_content_vect.toarray())
print(vectorizer.get_feature_names_out())
print(new_content_vect)


[[1 1 0 1 0 0 1 1 0 1 0]]
['날씨' '덥다' '맑음' '매우' '먹지' '목요일' '서울' '오늘' '이다' '점심' '좋다']
  (0, 0)	1
  (0, 1)	1
  (0, 3)	1
  (0, 6)	1
  (0, 7)	1
  (0, 9)	1


In [43]:
# 샘플 데이터와 훈련 데이터 거리 확인

import scipy as sp

#거리 구해주는 함수
def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

best_doc = None
best_dist = 65535
best_i = None

In [46]:
for i in range(0, len(contents)):
    post_vec = X.getrow(i)
    d = dist_raw(post_vec, new_content_vect)
    print("== %i 번째 문장과의 거리: %.2f : %s" %(i,d,contents[i]))
    if d<best_dist:
        best_dist = d
        best_i = i

== 0 번째 문장과의 거리: 2.00 : 오늘은 날씨가 매우 좋다
== 1 번째 문장과의 거리: 1.73 : 오늘도 날씨가 덥다
== 2 번째 문장과의 거리: 2.65 : 오늘은 목요일이다
== 3 번째 문장과의 거리: 2.24 : 서울 날씨는 맑음
== 4 번째 문장과의 거리: 2.24 : 오늘 점심 뭐 먹지
